## Reading in Data & Importing Lib

### Set up Dir

In [ ]:
import sys, os
if 'google.colab' in sys.modules:

    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/School stuff/Y4 S1/DSA4264'

    # move to Google Drive directory
    os.chdir(path_to_file)

### Downloads

### Import Lib

In [ ]:
%pip install bertopic

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px
import plotly.graph_objects as go

import re,nltk
import string
import gensim

from transformers import AutoTokenizer
from transformers import pipeline
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from plotly.subplots import make_subplots
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

### Read in Data

In [ ]:
Data1 = pd.read_csv('../data/Data_1.csv', engine = 'python') 
Data2 = pd.read_csv('../data/Data_2.csv', engine = 'python')
Data3 = pd.read_csv('../data/Data_3.csv', engine = 'python')
Data4 = pd.read_csv('../data/Data_4.csv', engine = 'python')
Data5 = pd.read_csv('../data/Data_5.csv', engine = 'python')
Data6 = pd.read_csv('../data/Data_6.csv', engine = 'python')
Data7 = pd.read_csv('../data/Data_7.csv', engine = 'python')

### Merge Data

In [ ]:
Data = pd.concat([Data1, Data2, Data3, Data4, Data5, Data6, Data7], ignore_index=True, axis=0)

Data = Data.drop(columns=['Unnamed: 0'])

In [ ]:
del Data1, Data2, Data3, Data4, Data5, Data6, Data7

### Datatypes

In [ ]:
Data['timestamp']= pd.to_datetime(Data['timestamp'])
Data['post_timestamp']= pd.to_datetime(Data['timestamp'])

## BERT Topics

Repeat for r/Singapore, r/SingaporeRaw, r/SingaporeHappenings

In [ ]:
# Exclude specific post_titles for r/Singapore
exclude_condition = Data['post_title'].str.startswith('/r/singapore random discussion and small questions thread for').fillna(False)
Data_filtered = Data[~((Data['subreddit'] == 'r/Singapore') & exclude_condition)]

# Filter for subreddit == r/SingaporeHappenings
singapore_posts = Data_filtered[Data_filtered['subreddit'] == 'r/SingaporeHappenings']

# Get unique post_title 
unique_post_titles = singapore_posts['post_title'].dropna().unique()

# Initialize BERTopic
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model, language="english")

# Fit BERTopic on the unique post titles
topics, probabilities = topic_model.fit_transform(unique_post_titles)

# Create a DataFrame with the results for the unique post titles
topic_df = pd.DataFrame({
    'post_title': unique_post_titles,
    'topic': topics,
    'topic_probability': probabilities
})


def get_topic_words(topic):
    return ' '.join([word for word, _ in topic_model.get_topic(topic, full=True)['Main']][:4])

topic_df['topic_words'] = topic_df['topic'].apply(get_topic_words)

In [ ]:
topic_df.to_csv('../data/Data.csv', encoding = 'utf-8-sig')
files.download('../data/Data.csv')